In [1]:
import pandas as pd
import numpy as np

In [2]:
# Среди респондентов General Social Survey 2014 года хотя бы раз в месяц проводят вечер в баре 203 женщины и 239 мужчин;
# реже, чем раз в месяц, это делают 718 женщин и 515 мужчин.
# Посчитайте значение коэффициента корреляции Мэтьюса между полом и частотой похода в бары.
# Округлите значение до трёх знаков после десятичной точки.

In [3]:
def Matthews_correlation(a, b, c, d):
    return (a*d-b*c)/np.sqrt((a+b)*(a+c)*(b+d)*(c+d))

In [4]:
men_visit_monthly = 239
women_visit_monthly = 203
men_visit_rarely = 515
women_visit_rarely = 718
print ("Matthews correlation is: %.3f" % np.round(Matthews_correlation(men_visit_monthly, men_visit_rarely, women_visit_monthly, women_visit_rarely), 3))

Matthews correlation is: 0.109


In [5]:
# В предыдущей задаче проверьте, значимо ли коэффициент корреляции Мэтьюса отличается от нуля.
# Посчитайте достигаемый уровень значимости; используйте функцию scipy.stats.chi2_contingency.

In [6]:
from scipy.stats import chi2_contingency
chi_stat = chi2_contingency([[men_visit_monthly, men_visit_rarely], [women_visit_monthly, women_visit_rarely]])
print ("Chi2 p-value is: %f" % chi_stat[1])

Chi2 p-value is: 0.000011


In [7]:
# В предыдущей задаче давайте попробуем ответить на немного другой вопрос: отличаются ли доля мужчин и доля женщин, относительно часто проводящих вечера в баре?
# Постройте 95% доверительный интервал для разности долей, вычитая долю женщин из доли мужчин.
# Чему равна его нижняя граница? Округлите до четырёх знаков после десятичной точки.

In [8]:
from scipy.stats import norm
def proportions_confint_diff_ind(p1, count1, p2, count2, alpha = 0.05):    
    z = norm.ppf(1 - alpha / 2.)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1. - p1)/ count1 + p2 * (1 - p2)/ count2)
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1. - p1)/ count1 + p2 * (1 - p2)/ count2)
    
    return (left_boundary, right_boundary)

In [9]:
all_men = men_visit_monthly + men_visit_rarely
all_women = women_visit_monthly + women_visit_rarely
men_proportion = float(men_visit_monthly)/float(all_men)
women_proportion = float(women_visit_monthly)/float(all_women)
conf_interval_diff = proportions_confint_diff_ind(men_proportion, all_men, women_proportion, all_women)
print ("Interval for men and women bar visitors proportion diff: [%.4f, %.4f]" % (np.round(conf_interval_diff[0],4),np.round(conf_interval_diff[1],4)))

Interval for men and women bar visitors proportion diff: [0.0539, 0.1392]


In [10]:
# Проверьте гипотезу о равенстве долей любителей часто проводить вечера в баре среди мужчин и женщин.
# Посчитайте достигаемый уровень значимости, используя двустороннюю альтернативу.

In [11]:
def proportions_diff_z_stat_ind(p1, count1, p2, count2, alpha = 0.05):
    P = float(p1*count1 + p2*count2) / (count1 + count2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / count1 + 1. / count2))

In [12]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - norm.cdf(z_stat)

In [13]:
pvalue = proportions_diff_z_test(proportions_diff_z_stat_ind(men_proportion, all_men, women_proportion, all_women))
print ("p-value for men-women proportion difference is: %f" % pvalue)

p-value for men-women proportion difference is: 0.000008
